## FEATURE ENGINERRING and normalization

For Business:
- ATTRIBUTES
- CATEGORIES
- HOURS

In [3]:
## Import Libraries ##
import pandas as pd
import datetime
import ast
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [21]:
business2 = pd.read_json('./data/business.json', lines = True)

In [ ]:
business2.head()

In [22]:
atributtes = pd.json_normalize(data = business2['attributes'])

In [7]:
atributtes['BusinessParking'].fillna("{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}", inplace=True)
atributtes.loc[atributtes['BusinessParking'] == 'None', 'BusinessParking'] = "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"

atributtes.loc[atributtes['Ambience'] == 'None', 'Ambience'] = "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}"
atributtes['Ambience'] = atributtes['Ambience'].fillna("{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}")

In [ ]:
def dicc(row):
    """
    It takes a string that looks like a dictionary and converts it to a dictionary.
    
    :param row: The row of data that you want to convert to a dictionary
    :return: A dictionary
    """
    #result = json.loads(row)
    result = ast.literal_eval(row)
    return result

In [ ]:
garage = pd.json_normalize(atributtes.BusinessParking.apply(dicc))
ambience = pd.json_normalize(atributtes['Ambience'].apply(dicc))

In [ ]:
garage.head()

In [ ]:
atributtes['garage'] = garage['garage']
atributtes['garage'].fillna(0, inplace=True)

In [ ]:
atributtes.drop(['BusinessParking', 'Ambience'], axis=1, inplace=True)

In [ ]:
ambience['good_ambience'] = ambience['romantic'] + ambience['intimate'] + ambience['classy'] + ambience['hipster'] + ambience['touristy'] + ambience['trendy'] + ambience['upscale'] + ambience['casual'] + ambience['divey']

In [ ]:
ambience.loc[ambience['good_ambience'] > 1, 'good_ambience'] = 1
ambience.fillna(0, inplace=True)

In [ ]:
ambience['good_ambience'].value_counts()

In [ ]:
atributtes['good_ambience'] = ambience['good_ambience']

In [ ]:
atributtes.head()

In [32]:
atributtes['RestaurantsTakeOut'].fillna(0, inplace=True)
atributtes['RestaurantsDelivery'].fillna(0, inplace=True)
atributtes['RestaurantsTakeOut'] = atributtes['RestaurantsTakeOut'].map({'True': 1, 'False': 0, 'None': 0})
atributtes['RestaurantsDelivery'] = atributtes['RestaurantsDelivery'].map({'True': 1, 'False': 0, 'None': 0})
atributtes['delivery'] = atributtes['RestaurantsTakeOut'] + atributtes['RestaurantsDelivery']
atributtes['delivery'] = pd.to_numeric(atributtes['delivery'], errors='coerce').fillna(0).astype(int)

In [33]:
atributtes['delivery'].unique()

array([0])

In [128]:
atributtes.notna().sum().sort_values(ascending=False).head(20)

good_ambience                 150346
garage                        150346
BusinessAcceptsCreditCards    119765
RestaurantsPriceRange2         85314
BikeParking                    72638
RestaurantsTakeOut             59857
WiFi                           56914
RestaurantsDelivery            56282
delivery                       54628
GoodForKids                    53375
OutdoorSeating                 48802
RestaurantsReservations        45247
HasTV                          45084
RestaurantsGoodForGroups       44170
Alcohol                        43189
ByAppointmentOnly              42339
Caters                         40127
RestaurantsAttire              39255
NoiseLevel                     37993
GoodForMeal                    29087
dtype: int64

In [129]:
top20 = atributtes.notna().sum().sort_values(ascending=False).head(20).index.tolist()

In [ ]:
atributtes['RestaurantsPriceRange2'] = pd.to_numeric(atributtes['RestaurantsPriceRange2'])

In [130]:
atributtes[top20].head()

,good_ambience,garage,BusinessAcceptsCreditCards,RestaurantsPriceRange2,BikeParking,RestaurantsTakeOut,WiFi,RestaurantsDelivery,delivery,GoodForKids,OutdoorSeating,RestaurantsReservations,HasTV,RestaurantsGoodForGroups,Alcohol,ByAppointmentOnly,Caters,RestaurantsAttire,NoiseLevel,GoodForMeal
0,0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
1,0,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,False,True,2,True,False,u'no',False,FalseFalse,NaN,False,False,False,NaN,NaN,False,False,NaN,NaN,NaN
3,0,False,False,1,True,True,u'free',False,TrueFalse,NaN,False,NaN,NaN,NaN,u'none',False,True,NaN,NaN,NaN
4,0,0,True,NaN,True,True,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN


In [131]:
atributtes = atributtes[top20]

In [ ]:
atributtes.fillna(0, inplace=True)
atributtes.replace('None', 0, inplace=True)
atributtes.replace('False', 0, inplace=True)
atributtes.replace(False, 0, inplace=True)
atributtes.replace('True', 1, inplace=True)

In [ ]:
atributtes['delivery'] = atributtes['RestaurantsTakeOut'] + atributtes['RestaurantsDelivery']
atributtes.drop(['RestaurantsTakeOut', 'RestaurantsDelivery'], axis=1, inplace=True)

In [ ]:

atributtes['WiFi'] = np.where(atributtes['WiFi'] != 0, 1, 0)

In [140]:
atributtes.loc[atributtes['Alcohol'] == "u'none'", 'Alcohol'] = 0
atributtes.loc[atributtes['Alcohol'] == "'none'", 'Alcohol'] = 0
atributtes.loc[atributtes['Alcohol'] != 0, 'Alcohol'] = 1
atributtes['Alcohol'].unique()

array([0, 1], dtype=object)

In [173]:
atributtes.dtypes == 'object'

good_ambience                 False
garage                         True
BusinessAcceptsCreditCards    False
RestaurantsPriceRange2         True
BikeParking                   False
WiFi                          False
delivery                      False
GoodForKids                   False
OutdoorSeating                False
RestaurantsReservations       False
HasTV                         False
RestaurantsGoodForGroups      False
Alcohol                        True
ByAppointmentOnly             False
Caters                        False
RestaurantsAttire              True
NoiseLevel                     True
GoodForMeal                    True
dtype: bool

In [176]:
atributtes.loc[:,atributtes.dtypes == 'object'].sample(10)

,garage,RestaurantsPriceRange2,Alcohol,RestaurantsAttire,NoiseLevel,GoodForMeal
96113,0,1,0,0,0,0
45258,0,0,0,0,0,0
70874,0,0,0,0,0,0
142443,0,0,0,0,0,0
86802,0,0,0,0,0,0
117205,0,2,0,0,0,0
48236,0,2,1,1,2,"{'dessert': False, 'latenight': False, 'lunch'..."
30574,0,2,0,0,0,0
145223,0,1,0,1,2,0
131223,0,2,0,0,0,0


In [164]:
atributtes.loc[atributtes['NoiseLevel'].str.contains('quiet').fillna(False), 'NoiseLevel'] = 1
atributtes.loc[atributtes['NoiseLevel'].str.contains('average').fillna(False), 'NoiseLevel'] = 2
atributtes.loc[atributtes['NoiseLevel'].str.contains('loud').fillna(False), 'NoiseLevel'] = 3

In [167]:
atributtes.loc[atributtes['RestaurantsAttire'] != 0,'RestaurantsAttire'] = 1

In [175]:
atributtes.loc[atributtes['garage'] != 0,'garage'] = 1

In [ ]:
atributtes.loc[atributtes['GoodForMeal'] == 0,'GoodForMeal'] = "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}"
atributtes['GoodForMeal'].fillna("{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}", inplace=True)

In [195]:
meal_types = pd.json_normalize(atributtes['GoodForMeal'].apply(dicc))

In [199]:
meal_types.describe()

,dessert,latenight,lunch,dinner,brunch,breakfast
count,145753,146602,147951,148994,146748,147887
unique,2,2,2,2,2,2
top,False,False,False,False,False,False
freq,143975,145271,133735,135896,143872,144855


In [ ]:
atributtes.drop(['GoodForMeal'], axis=1, inplace=True)
atributtes['meal_diversity'] = meal_types['dessert'] + meal_types['latenight'] + meal_types['lunch'] + meal_types['dinner'] + meal_types['brunch'] + meal_types['breakfast']

In [ ]:
atributtes['meal_diversity'].fillna(0, inplace=True)

In [205]:
atributtes.isna().sum().sort_values(ascending=False).head(20)

good_ambience                 0
garage                        0
NoiseLevel                    0
RestaurantsAttire             0
Caters                        0
ByAppointmentOnly             0
Alcohol                       0
RestaurantsGoodForGroups      0
HasTV                         0
RestaurantsReservations       0
OutdoorSeating                0
GoodForKids                   0
delivery                      0
WiFi                          0
BikeParking                   0
RestaurantsPriceRange2        0
BusinessAcceptsCreditCards    0
meal_diversity                0
dtype: int64

In [ ]:
atributtes['business_id'] = business2['business_id']
atributtes.to_csv('./data/atributtes_clean.csv', index=False)

## HOURS

In [ ]:
hours = pd.json_normalize(business2['hours'])
hours['business_id'] = business2['business_id']

In [304]:
hours.sample(5)

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,business_id
98069,7:0-19:0,7:0-19:0,7:0-19:0,7:0-19:0,7:0-19:0,7:0-17:0,NaN,S5VCmo_NWpdYneAE0p93PA
95472,0:0-0:0,8:0-18:0,8:0-18:0,8:0-18:0,8:0-18:0,9:0-17:0,NaN,oS5I81wVdiD_yBNALMj64w
117543,9:0-18:0,9:0-18:0,9:0-19:0,9:0-18:0,9:0-18:0,8:30-13:0,NaN,Mro39taLejqtvsitvMCfGg
54846,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-15:0,NaN,YURjQmTSH2Nci8tmegOMbg
25790,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,IpfGcFwwCAh9Si_kjsoPhg


In [305]:
subset = hours[~pd.isnull(hours[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]).any(1)].index
hours['7days'] = 0
hours.loc[subset, '7days'] = 1

In [306]:
subset = hours[~pd.isnull(hours[['Friday', 'Saturday', 'Sunday']]).any(1)].index
hours['weekends'] = 0
hours.loc[subset, 'weekends'] = 1

In [307]:
def open_time(x):
    """
    If the value is null, return null, otherwise return the first value in the list of values separated
    by a dash
    
    :param x: the value of the cell in the column
    :return: The first element of the list that is returned by the split() method.
    """
    if pd.isnull(x):
        return None
    else:
        return x.split('-')[0]

def close_time(x):
    """
    If the value is null, return null, otherwise return the second value in the list
    
    :param x: the value of the cell in the column
    :return: The close time of the restaurant
    """
    if pd.isnull(x):
        return None
    else:
        return x.split('-')[1]

In [308]:
hours['monday_open_time'] = pd.to_datetime(hours['Monday'].apply(open_time), format='%H:%M')
hours['monday_close_time'] = pd.to_datetime(hours['Monday'].apply(close_time), format='%H:%M')
hours['monday_total_hours'] = (hours['monday_close_time'] - hours['monday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['monday_total_hours'] == 0.0, 'monday_total_hours'] = 24.0

hours['tuesday_open_time'] = pd.to_datetime(hours['Tuesday'].apply(open_time), format='%H:%M')
hours['tuesday_close_time'] = pd.to_datetime(hours['Tuesday'].apply(close_time), format='%H:%M')
hours['tuesday_total_hours'] = (hours['tuesday_close_time'] - hours['tuesday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['tuesday_total_hours'] == 0.0, 'tuesday_total_hours'] = 24.0


hours['wednesday_open_time'] = pd.to_datetime(hours['Wednesday'].apply(open_time), format='%H:%M')
hours['wednesday_close_time'] = pd.to_datetime(hours['Wednesday'].apply(close_time), format='%H:%M')
hours['wednesday_total_hours'] = (hours['wednesday_close_time'] - hours['wednesday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['wednesday_total_hours'] == 0.0, 'wednesday_total_hours'] = 24.0


hours['thursday_open_time'] = pd.to_datetime(hours['Thursday'].apply(open_time), format='%H:%M')
hours['thursday_close_time'] = pd.to_datetime(hours['Thursday'].apply(close_time), format='%H:%M')
hours['thursday_total_hours'] = (hours['thursday_close_time'] - hours['thursday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['thursday_total_hours'] == 0.0, 'thursday_total_hours'] = 24.0


hours['friday_open_time'] = pd.to_datetime(hours['Friday'].apply(open_time), format='%H:%M')
hours['friday_close_time'] = pd.to_datetime(hours['Friday'].apply(close_time), format='%H:%M')
hours['friday_total_hours'] = (hours['friday_close_time'] - hours['friday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['friday_total_hours'] == 0.0, 'friday_total_hours'] = 24.0


hours['saturday_open_time'] = pd.to_datetime(hours['Saturday'].apply(open_time), format='%H:%M')
hours['saturday_close_time'] = pd.to_datetime(hours['Saturday'].apply(close_time), format='%H:%M')
hours['saturday_total_hours'] = (hours['saturday_close_time'] - hours['saturday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['saturday_total_hours'] == 0.0, 'saturday_total_hours'] = 24.0


hours['sunday_open_time'] = pd.to_datetime(hours['Sunday'].apply(open_time), format='%H:%M')
hours['sunday_close_time'] = pd.to_datetime(hours['Sunday'].apply(close_time), format='%H:%M')
hours['sunday_total_hours'] = (hours['sunday_close_time'] - hours['sunday_open_time'])/pd.Timedelta(hours=1)
hours.loc[hours['sunday_total_hours'] == 0.0, 'sunday_total_hours'] = 24.0


In [309]:
lista_total_open_hours = ['monday_total_hours', 'tuesday_total_hours', 'wednesday_total_hours', 'thursday_total_hours', 'friday_total_hours', 'saturday_total_hours', 'sunday_total_hours']
lista_open = ['monday_open_time', 'tuesday_open_time', 'wednesday_open_time', 'thursday_open_time', 'friday_open_time', 'saturday_open_time', 'sunday_open_time']
lista_close = ['monday_close_time', 'tuesday_close_time', 'wednesday_close_time', 'thursday_close_time', 'friday_close_time', 'saturday_close_time', 'sunday_close_time']
lista_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
lista_total =  lista_total_open_hours + lista_open + lista_close + lista_days

In [310]:
hours['n_open_days'] = pd.notnull(hours[lista_open]).sum(axis=1)

In [311]:
hours['mean_total_hours_open'] = hours[lista_total_open_hours].abs().mean(axis=1)

In [312]:
hours['mean_total_hours_open'].describe()

count    127123.000000
mean         10.772699
std           4.154365
min           0.016667
25%           8.428571
50%          10.000000
75%          11.857143
max          24.000000
Name: mean_total_hours_open, dtype: float64

In [313]:
def normalize_hours(lista:list, df):
    """
    It takes a list of columns and a dataframe as input, and for each column in the list, it converts
    the column to datetime, and then subtracts the datetime from the datetime.normalize() of the same
    column
    
    :param lista: list of columns to be normalized
    :type lista: list
    :param df: the dataframe
    """
    for i in lista:
        df.loc[pd.notnull(df[i]), i] = pd.to_datetime(df.loc[pd.notnull(df[i]), i]) - pd.to_datetime(df.loc[pd.notnull(df[i]), i]).dt.normalize()

def mean_open_hour(lista:list, df):
    """
    It takes a list of columns and a dataframe as input, and for each row in the dataframe, it
    calculates the mean of the values in the columns in the list, and stores the result in a new column
    called 'mean_open_hour'.
    
    :param lista: list of columns to be averaged
    :type lista: list
    :param df: the dataframe
    """
    for i in range(df.shape[0]):
        temp_sum = []
        for j in lista:
            if pd.notnull(df.loc[i, j]):
                temp_sum.append(df.loc[i, j])
        if len(temp_sum) > 0:
            df.loc[i, 'mean_open_hour'] = sum(temp_sum,datetime.timedelta())/len(temp_sum)

    """
    For each row in the dataframe, if the value in the column is not null, then add it to a list. If the
    list is not empty, then take the average of the list and add it to a new column in the dataframe.
    
    :param lista: list of columns to be averaged
    :type lista: list
    :param df: the dataframe
    """
def mean_close_hour(lista:list, df):
    for i in range(df.shape[0]):
        temp_sum = []
        for j in lista:
            if pd.notnull(df.loc[i, j]):
                temp_sum.append(df.loc[i, j])
        if len(temp_sum) > 0:
            df.loc[i, 'mean_close_hour'] = sum(temp_sum,datetime.timedelta())/len(temp_sum)

In [314]:
normalize_hours(lista_open, hours)
normalize_hours(lista_close, hours)
mean_open_hour(lista_open, hours)
mean_close_hour(lista_close, hours)

In [315]:
hours.drop(lista_total, axis=1, inplace=True)

In [316]:
hours.head()

,business_id,7days,weekends,n_open_days,mean_total_hours_open,mean_open_hour,mean_close_hour
0,Pns2l4eNsfO8kk83dixA6A,0,0,0,NaN,NaT,NaT
1,mpf3x-BjTdTEA3yCZrAYPw,0,0,6,12.000000,0 days 06:40:00,0 days 14:40:00
2,tUFrWirKiKi_TAnsVWINQQ,1,1,7,14.285714,0 days 08:00:00,0 days 22:17:08.571428571
3,MTSW4McQd7CbVtyjqoe9mw,1,1,7,13.428571,0 days 07:00:00,0 days 20:25:42.857142857
4,mWMc6_wTdE0EUBKIGXDVfA,0,1,5,8.000000,0 days 13:12:00,0 days 21:12:00


In [317]:
hours.describe()

,7days,weekends,n_open_days,mean_total_hours_open,mean_open_hour,mean_close_hour
count,150346.000000,150346.000000,150346.000000,127123.000000,127123,127123
mean,0.474146,0.534115,5.327811,10.772699,0 days 08:52:15.181635109,0 days 16:06:55.853130994
std,0.499333,0.498836,2.445235,4.154365,0 days 03:14:14.536837022,0 days 06:06:27.727208463
min,0.000000,0.000000,0.000000,0.016667,0 days 00:00:00,0 days 00:00:00
25%,0.000000,0.000000,5.000000,8.428571,0 days 07:30:00,0 days 15:25:42.857142857
50%,0.000000,1.000000,6.000000,10.000000,0 days 09:00:00,0 days 17:36:00
75%,1.000000,1.000000,7.000000,11.857143,0 days 10:30:00,0 days 20:00:00
max,1.000000,1.000000,7.000000,24.000000,0 days 23:30:00,0 days 23:59:00


In [318]:
hours.to_csv('hours_fixed.csv', index=False)

In [233]:
business2['categories'].str.split(', ', expand=True).stack().value_counts().head(10).index.tolist()

['Restaurants',
 'Food',
 'Shopping',
 'Home Services',
 'Beauty & Spas',
 'Nightlife',
 'Health & Medical',
 'Local Services',
 'Bars',
 'Automotive']

In [246]:
categories = business2['categories'].str.split(', ', expand=True)

In [247]:
categories.index = business2['business_id']

In [248]:
df = categories.T.stack().groupby('business_id').apply(list).reset_index(name='categories')

In [249]:
df

,business_id,categories
0,Pns2l4eNsfO8kk83dixA6A,"[Doctors, Traditional Chinese Medicine, Naturo..."
1,mpf3x-BjTdTEA3yCZrAYPw,"[Shipping Centers, Local Services, Notaries, M..."
2,tUFrWirKiKi_TAnsVWINQQ,"[Department Stores, Shopping, Fashion, Home & ..."
3,MTSW4McQd7CbVtyjqoe9mw,"[Restaurants, Food, Bubble Tea, Coffee & Tea, ..."
4,mWMc6_wTdE0EUBKIGXDVfA,"[Brewpubs, Breweries, Food]"
...,...,...
150238,IUQopTMmYQG-qRtBk-8QnA,"[Nail Salons, Beauty & Spas]"
150239,c8GjPIOTGVmIemT7j5_SyQ,"[Pets, Nurseries & Gardening, Pet Stores, Hobb..."
150240,_QAMST-NrQobXduilWEqSw,"[Shopping, Jewelry, Piercing, Toy Stores, Beau..."
150241,mtGm22y5c2UHNXDFAjaPNw,"[Fitness/Exercise Equipment, Eyewear & Opticia..."


In [250]:
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('categories')),
                          columns=mlb.classes_,
                          index=df.index))

In [ ]:
df['total_categories'] = df.sum(axis=1)

In [255]:
most_frequent = ['Restaurants',
 'Food',
 'Shopping',
 'Home Services',
 'Beauty & Spas',
 'Nightlife',
 'Health & Medical',
 'Local Services',
 'Bars',
 'Automotive', 'total_categories', 'business_id']

In [257]:
df[most_frequent].sort_values(by='total_categories', ascending=False).head(10)

,Restaurants,Food,Shopping,Home Services,Beauty & Spas,Nightlife,Health & Medical,Local Services,Bars,Automotive,total_categories,business_id
143059,1,1,1,0,0,0,0,0,0,0,36,ytynqOUb3hjKeJfRj5Tshw
86993,0,0,1,0,1,1,0,1,1,0,26,TfRyyMfqDxtC1KA1jLkmvg
106365,1,1,0,0,0,1,0,0,1,0,25,5hyIty7KkRfVE_1gCKbUvw
86376,0,0,1,1,0,0,0,1,0,0,24,mmlZz6ypFdgHROjt2AnSsw
3086,1,0,1,0,0,0,1,1,0,1,23,kdboJGRQZXuKYLIFMvsoJw
136920,0,0,1,1,0,0,0,1,0,1,23,Zk4aemT88EwIvoa-19z4mg
57574,1,1,1,0,0,1,0,0,1,0,22,iOFbMitGdfU6i5xm4nZ8Ww
100045,1,1,0,0,0,1,0,0,1,0,22,4fKGv-3XMlxv_vuiFbuwLA
102462,1,1,0,0,0,1,0,0,0,0,22,lPday_z2XpRJqbFWyCh6dw
109339,1,1,1,0,0,0,0,0,0,0,21,-mDAFyX5kZHXhxjELeMhiQ


In [259]:
df[most_frequent].to_csv('./data/categories_fixed.csv', index=False)